In [1]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from typing import List, Dict, Tuple
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import warnings
import cupy as cp
warnings.filterwarnings("ignore")

Duplicate key in file WindowsPath('d:/source/Anaconda/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 258 ('font.family:  Microsoft JhengHei')


In [22]:
data = pd.read_csv("datas/5min_MYM_只用IB量價資料_regression.csv")

In [2]:
data = pd.read_csv("datas/5min_MYM_量價+YM+ES+VIX資料_class.csv")

下面這個不要動，是在觀察漲跌標準差用的

In [23]:
#觀察漲跌的標準差是多少，用來決定class的門檻值
data[
    [
        "future_5min_change",
        "future_10min_change",
        "future_15min_change",
        "future_30min_change",
        "future_60min_change",
        "future_90min_change",
        "future_120min_change",
        "future_180min_change",
        "future_240min_change",
    ]
].describe()

,future_5min_change,future_10min_change,future_15min_change,future_30min_change,future_60min_change,future_90min_change,future_120min_change,future_180min_change,future_240min_change
count,18712.000000,18707.000000,18702.000000,18687.000000,18657.000000,18627.000000,18597.000000,18537.000000,18477.000000
mean,0.002100,0.004154,0.006206,0.012331,0.023995,0.035968,0.047988,0.072643,0.098254
std,0.137146,0.190865,0.232627,0.322724,0.448382,0.550104,0.641025,0.797345,0.929750
min,-1.774301,-1.746236,-1.753214,-2.177889,-2.276888,-2.900994,-2.764514,-4.076634,-4.893189
25%,-0.050397,-0.071292,-0.087100,-0.128220,-0.199788,-0.250610,-0.278806,-0.322662,-0.342064
50%,0.002587,0.007081,0.009454,0.019969,0.038950,0.060525,0.080980,0.112165,0.147506
75%,0.060851,0.088304,0.109730,0.171596,0.260453,0.337419,0.427545,0.604783,0.721625
max,0.977869,1.062134,1.340572,1.549550,1.962723,2.280553,2.254635,2.385865,2.768850


In [3]:
data_df=data.copy()
data_df['datetime']=pd.to_datetime(data_df['datetime'])
# 移除15:00~17:00以及03:00~04:00的資料
data_df_清洗 = data_df[
    (data_df["datetime"].dt.time >= pd.to_datetime("17:00:00").time())
    | (data_df["datetime"].dt.time <= pd.to_datetime("03:00:00").time())
]
# data.drop(["datetime"], axis=1, inplace=True)

numeric_cols = data_df_清洗.select_dtypes(include=["number"]).columns
filtered_rows = data_df_清洗[(data_df_清洗[numeric_cols] > 999999999).any(axis=1)]
data_df_清洗 = filtered_rows.dropna()

# 13873 row
df_train = data_df_清洗.iloc[:10000]
df_validation = data_df_清洗.iloc[10000:11000]
df_predict = data_df_清洗.iloc[11000:11100]

In [4]:
for col in data_df_清洗.columns:
    print(f"'{col}',")
# print(list(data_df_清洗.columns))

# pd.DataFrame(data_df_清洗.columns, columns=["Column Names"])

'Unnamed: 0',
'datetime',
'future_5min_change',
'future_10min_change',
'future_15min_change',
'future_30min_change',
'future_60min_change',
'future_90min_change',
'future_120min_change',
'future_180min_change',
'future_240min_change',
'open',
'high',
'low',
'close',
'volume',
'mym_ma_15',
'mym_ma_slope_15',
'mym_ma_30',
'mym_ma_slope_30',
'mym_ma_60',
'mym_ma_slope_60',
'mym_ma_120',
'mym_ma_slope_120',
'mym_ma_360',
'mym_ma_slope_360',
'mym_ma_1440',
'mym_ma_slope_1440',
'mym_ma_7200',
'mym_ma_slope_7200',
'mym_volume_ma_15',
'mym_volume_ma_slope_15',
'mym_volume_ma_30',
'mym_volume_ma_slope_30',
'mym_volume_ma_60',
'mym_volume_ma_slope_60',
'mym_volume_ma_120',
'mym_volume_ma_slope_120',
'mym_volume_ma_360',
'mym_volume_ma_slope_360',
'mym_volume_ma_720',
'mym_volume_ma_slope_720',
'mym_volume_ma_1440',
'mym_volume_ma_slope_1440',
'mym_volume_ma_7200',
'mym_volume_ma_slope_7200',
'vwap_cumulative_volume',
'vwap_cumulative_volume_price',
'VWAP',
'VWAP_30漲跌幅',
'VWAP_60漲跌幅',
'ATR_14',
'

In [11]:
# 假設 data 是您的 dataframe
# 先定義特徵和目標變數
features = [
    "open",
    "high",
    "low",
    "close",
    "volume",
    "mym_ma_15",
    "mym_ma_slope_15",
    "mym_ma_30",
    "mym_ma_slope_30",
    "mym_ma_60",
    "mym_ma_slope_60",
    "mym_ma_120",
    "mym_ma_slope_120",
    "mym_ma_360",
    "mym_ma_slope_360",
    "mym_ma_1440",
    "mym_ma_slope_1440",
    "mym_ma_7200",
    "mym_ma_slope_7200",
    "mym_volume_ma_15",
    "mym_volume_ma_slope_15",
    "mym_volume_ma_30",
    "mym_volume_ma_slope_30",
    "mym_volume_ma_60",
    "mym_volume_ma_slope_60",
    "mym_volume_ma_120",
    "mym_volume_ma_slope_120",
    "mym_volume_ma_360",
    "mym_volume_ma_slope_360",
    "mym_volume_ma_720",
    "mym_volume_ma_slope_720",
    "mym_volume_ma_1440",
    "mym_volume_ma_slope_1440",
    "mym_volume_ma_7200",
    "mym_volume_ma_slope_7200",
    "vwap_cumulative_volume",
    "vwap_cumulative_volume_price",
    "VWAP",
    "VWAP_30漲跌幅",
    "VWAP_60漲跌幅",
    "ATR_14",
    "KC_upper",
    "KC_lower",
    "ATR_21",
    "ATR_28",
    "ATR_42",
    "OBV",
    "14KD_lowest_low",
    "14KD_highest_high",
    "14%K",
    "14KD_%D",
    "28KD_lowest_low",
    "28KD_highest_high",
    "28%K",
    "28KD_%D",
    "RSI",
    "MACD",
    "Signal_Line",
    "MACD_Histogram",
    "BB_upper",
    "BB_lower",
    "return_15m",
    "high_return_15m",
    "low_return_15m",
    "return_30m",
    "high_return_30m",
    "low_return_30m",
    "return_60m",
    "high_return_60m",
    "low_return_60m",
    "return_120m",
    "high_return_120m",
    "low_return_120m",
    "return_240m",
    "high_return_240m",
    "low_return_240m",
    "minutes_from_open",
    "距離2130的時間差",
    "YM五分內最大1min成交量",
    "YM五分內成交總筆數",
    "ES close",
    "ES volume",
    "ES count",
    "VIX近 close",
    # "VIX近 volume",
    # "VIX近 count",
    "VIX_近 480分漲跌幅",
    "VIX_近 240分漲跌幅",
    "VIX_近 120分漲跌幅",
    "VIX_近 60分漲跌幅",
    "VIX_近 30分漲跌幅",
]
targets = [
    "future_5min_change",
    "future_10min_change",
    "future_15min_change",
    "future_30min_change",
    "future_60min_change",
    "future_90min_change",
    "future_120min_change",
    "future_180min_change",
    "future_240min_change",
]

##### 分類器 --可

In [10]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder


class TimeSeriesClassifier:
    def __init__(self, window_size=10):
        """
        初始化時間序列分類器
        """
        self.window_size = window_size
        self.model = None
        self.features = None
        self.label_encoder = LabelEncoder()

    def create_flattened_features(
        self, df, features, target, start_idx=None, end_idx=None
    ):
        if start_idx is None:
            start_idx = self.window_size
        if end_idx is None:
            end_idx = len(df)

        flattened_data = []
        flattened_labels = []

        for i in range(start_idx, end_idx):
            # 檢查時間間隔是否連續
            if df['datetime'].iloc[i] - df['datetime'].iloc[i - 1] != pd.Timedelta(minutes=5):
                continue  # 如果不連續，跳過該行

            historical_data = df[features].iloc[i - self.window_size: i]
            flattened_sample = historical_data.values.flatten()
            flattened_data.append(flattened_sample)
            flattened_labels.append(df[target].iloc[i])

        X_flattened = np.array(flattened_data)
        y_flattened = self.label_encoder.fit_transform(flattened_labels)

        return X_flattened, y_flattened


    def train(self, df, features, target, val_start_idx=None):
        self.features = features

        if val_start_idx is None:
            val_start_idx = int(len(df) * 0.8)

        X_train, y_train = self.create_flattened_features(
            df, features, target, start_idx=self.window_size, end_idx=val_start_idx
        )

        X_val, y_val = self.create_flattened_features(
            df, features, target, start_idx=val_start_idx, end_idx=len(df)
        )

        n_classes = len(np.unique(y_train))

        params = {
            "objective": "multiclass",
            "num_class": n_classes,
            "metric": "multi_logloss",
            "boosting_type": "gbdt",
            "learning_rate": 0.01,
            "num_leaves": 40,
            "verbose": -1,
            "colsample_bytree": 0.8,
            "subsample": 0.8,
            "min_child_samples": 40,
        }

        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

        self.model = lgb.train(
            params,
            train_data,
            valid_sets=[train_data, val_data],
            callbacks=[
                lgb.log_evaluation(period=0),
                lgb.early_stopping(stopping_rounds=30),
            ],
        )

        # Display feature importance after training
        self.display_feature_importance()

        return self

    def display_feature_importance(self):
        """
        顯示特徵重要性
        """
        if self.model is None:
            raise ValueError("模型尚未訓練，請先調用train方法")

        # 取得特徵重要性
        feature_importance = self.model.feature_importance(importance_type='gain')  # 或 'split'
        feature_names = self.features * self.window_size  # 每個特徵都會展開 window_size 次

        # 將特徵重要性與特徵名稱轉換為 DataFrame
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': feature_importance
        })

        # 顯示按重要性排序的特徵
        importance_df = importance_df.groupby('feature')['importance'].sum().sort_values(ascending=False)
        print("Feature Importance:")
        print(importance_df)
        importance_df.to_csv("datas/模型報告/8.跳過時間斷點.csv")
        return importance_df
    def validate_step(self, df, target, start_idx, end_idx):
        y_true = []
        y_pred = []

        for i in range(start_idx, end_idx):
            # 檢查時間間隔是否連續
            if df['datetime'].iloc[i] - df['datetime'].iloc[i - 1] != pd.Timedelta(minutes=5):
                continue  # 跳過非連續的時間點

            X_val = df[self.features].iloc[i - self.window_size: i].values.flatten().reshape(1, -1)
            current_true = df[target].iloc[i]
            pred_proba = self.model.predict(X_val)
            pred_class = np.argmax(pred_proba, axis=1)[0]
            predicted_label = self.label_encoder.inverse_transform([pred_class])[0]

            y_true.append(current_true)
            y_pred.append(predicted_label)

        return y_true, y_pred


    def validate(self, df, targets):
        """
        對多個目標進行驗證
        """
        results = {}

        for target in targets:
            y_true, y_pred = self.validate_step(
                df, target, start_idx=self.window_size, end_idx=len(df)
            )

            accuracy = accuracy_score(y_true, y_pred)
            class_report = classification_report(y_true, y_pred)

            print(f"\nValidation results for {target}:")
            print(f"Accuracy: {accuracy:.4f}")
            print("Detailed Classification Report:")
            print(class_report)

            results[target] = {
                "accuracy": accuracy,
                "classification_report": class_report,
                "predictions": y_pred,
            }

        return results

    def predict_step(self, df, start_idx, end_idx):
        """
        逐步預測，確保只使用歷史數據
        """
        predictions = []

        for i in range(start_idx, end_idx):
            # 只使用過去的數據
            X_test = (
                df[self.features]
                .iloc[i - self.window_size : i]
                .values.flatten()
                .reshape(1, -1)
            )

            # 預測並轉換回原始標籤
            pred_proba = self.model.predict(X_test)
            pred_class = np.argmax(pred_proba, axis=1)[0]
            predicted_label = self.label_encoder.inverse_transform([pred_class])[0]

            predictions.append(predicted_label)

        return predictions

    def predict(self, df, targets):
        """
        對多個目標進行預測
        """
        if self.model is None:
            raise ValueError("模型尚未訓練，請先調用train方法")

        predictions = {}

        for target in targets:
            pred_list = self.predict_step(
                df, start_idx=self.window_size, end_idx=len(df)
            )

            predictions[target] = pred_list

            # 如果有真實標籤，計算準確度
            if target in df.columns:
                true_values = df[target].iloc[self.window_size : len(df)].values
                accuracy = accuracy_score(true_values, pred_list)
                class_report = classification_report(true_values, pred_list)

                print(f"\nPrediction results for {target}:")
                print(f"Accuracy: {accuracy:.4f}")
                print("Detailed Classification Report:")
                print(class_report)

        return predictions

In [35]:
targets = [
    "future_5min_change",
    "future_10min_change",
    "future_15min_change",
    "future_30min_change",
    "future_60min_change",
    "future_90min_change",
    "future_120min_change",
    "future_180min_change",
    "future_240min_change",
]

In [12]:
# 使用示例
# 初始化分類器
classifier = TimeSeriesClassifier(window_size=10)

# 訓練模型
# 使用前80%的數據訓練，後20%驗證
train_end_idx = int(len(df_train) * 0.8)
target_n = 4
classifier.train(df_train, features, targets[target_n], val_start_idx=train_end_idx)

# 驗證模型
validation_results = classifier.validate(
    df_validation, targets[target_n : target_n + 1]
)

# 預測
# predictions = classifier.predict(df_predict, targets)
# +-0.05%
# 0:0.40
# 1:0.39
# 2:0.39
# 3:0.24
# 4:0.30
# 5:

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	training's multi_logloss: 0.677436	valid_1's multi_logloss: 0.703265
Feature Importance:
feature
OBV                       518.177103
mym_ma_7200               421.934097
vwap_cumulative_volume    420.751801
VIX_近 480分漲跌幅             390.091902
VWAP                      350.218307
                             ...    
low_return_30m              0.000000
low_return_60m              0.000000
mym_ma_120                  0.000000
mym_ma_30                   0.000000
距離2130的時間差                  0.000000
Name: importance, Length: 89, dtype: float64

Validation results for future_60min_change:
Accuracy: 0.5183
Detailed Classification Report:
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00       473
         1.0       0.52      1.00      0.68       509

    accuracy                           0.52       982
   macro avg       0.26      0.50      

##### 分類器調參數-CPU多執行緒

In [8]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import product
import pandas as pd


class TimeSeriesClassifier:
    def __init__(self, window_size=10):
        """
        初始化時間序列分類器
        """
        self.window_size = window_size
        self.model = None
        self.features = None
        self.label_encoder = LabelEncoder()
        self.best_params = None
        self.best_score = float("-inf")

    def create_flattened_features(
        self, df, features, target, start_idx=None, end_idx=None
    ):
        """
        創建扁平化特徵，可以指定開始和結束索引
        """
        if start_idx is None:
            start_idx = self.window_size
        if end_idx is None:
            end_idx = len(df)

        flattened_data = []
        flattened_labels = []

        for i in range(start_idx, end_idx):
            historical_data = df[features].iloc[i - self.window_size : i]
            flattened_sample = historical_data.values.flatten()
            flattened_data.append(flattened_sample)
            flattened_labels.append(df[target].iloc[i])

        X_flattened = np.array(flattened_data)
        y_flattened = self.label_encoder.fit_transform(flattened_labels)

        return X_flattened, y_flattened

    def _train_with_params(self, X_train, y_train, X_val, y_val, params):
        """
        使用指定參數訓練模型
        """
        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

        model = lgb.train(
            params,
            train_data,
            valid_sets=[train_data, val_data],
            callbacks=[
                lgb.log_evaluation(period=0),
                lgb.early_stopping(stopping_rounds=30),
            ],
        )

        # 使用驗證集評估模型
        val_pred = np.argmax(model.predict(X_val), axis=1)
        val_score = accuracy_score(y_val, val_pred)

        return model, val_score, params

    def tune_parameters(self, df, features, target, val_start_idx=None, n_jobs=-1):
        """
        使用多執行緒進行參數調優
        """
        self.features = features

        if val_start_idx is None:
            val_start_idx = int(len(df) * 0.8)

        # 準備訓練集和驗證集
        X_train, y_train = self.create_flattened_features(
            df, features, target, start_idx=self.window_size, end_idx=val_start_idx
        )

        X_val, y_val = self.create_flattened_features(
            df, features, target, start_idx=val_start_idx, end_idx=len(df)
        )

        n_classes = len(np.unique(y_train))

        # 定義參數網格
        param_grid = {
            "learning_rate": [0.01, 0.005, 0.001],
            "num_leaves": [31, 40, 50],
            "min_child_samples": [20, 30, 40],
            "subsample": [0.8, 0.9, 1.0],
            "colsample_bytree": [0.8, 0.9, 1.0],
        }

        # 生成所有參數組合
        param_combinations = [
            dict(zip(param_grid.keys(), v)) for v in product(*param_grid.values())
        ]

        # 為每個參數組合添加固定參數
        base_params = {
            "objective": "multiclass",
            "num_class": n_classes,
            "metric": "multi_logloss",
            "boosting_type": "gbdt",
            "verbose": -1,
        }

        for params in param_combinations:
            params.update(base_params)

        best_model = None
        results = []

        # 使用線程池進行並行訓練
        with ThreadPoolExecutor(max_workers=n_jobs if n_jobs > 0 else None) as executor:
            future_to_params = {
                executor.submit(
                    self._train_with_params, X_train, y_train, X_val, y_val, params
                ): params
                for params in param_combinations
            }

            for future in as_completed(future_to_params):
                model, score, params = future.result()
                results.append((score, params, model))

                if score > self.best_score:
                    self.best_score = score
                    self.best_params = params
                    best_model = model
                    print(f"\nNew best score: {score:.4f}")
                    print(
                        "Parameters:",
                        {k: v for k, v in params.items() if k not in base_params},
                    )

        # 使用最佳參數更新模型
        self.model = best_model

        # 將結果整理成DataFrame
        results_df = pd.DataFrame(
            [
                {
                    **{k: v for k, v in params.items() if k not in base_params},
                    "score": score,
                }
                for score, params, _ in results
            ]
        )

        return results_df

    def train(self, df, features, target, val_start_idx=None):
        """
        使用最佳參數訓練模型
        """
        if self.best_params is None:
            print("Warning: No tuned parameters found. Using default parameters.")
            return super().train(df, features, target, val_start_idx)

        self.features = features

        if val_start_idx is None:
            val_start_idx = int(len(df) * 0.8)

        X_train, y_train = self.create_flattened_features(
            df, features, target, start_idx=self.window_size, end_idx=val_start_idx
        )

        X_val, y_val = self.create_flattened_features(
            df, features, target, start_idx=val_start_idx, end_idx=len(df)
        )

        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

        self.model = lgb.train(
            self.best_params,
            train_data,
            valid_sets=[train_data, val_data],
            callbacks=[
                lgb.log_evaluation(period=0),
                lgb.early_stopping(stopping_rounds=30),
            ],
        )

        return self

    def validate_step(self, df, target, start_idx, end_idx):
        """
        逐步驗證，確保只使用歷史數據
        """
        if self.model is None:
            raise ValueError("模型尚未訓練，請先調用train方法")

        y_true = []
        y_pred = []

        for i in range(start_idx, end_idx):
            X_val = (
                df[self.features]
                .iloc[i - self.window_size : i]
                .values.flatten()
                .reshape(1, -1)
            )
            current_true = df[target].iloc[i]

            pred_proba = self.model.predict(X_val)
            pred_class = np.argmax(pred_proba, axis=1)[0]
            predicted_label = self.label_encoder.inverse_transform([pred_class])[0]

            y_true.append(current_true)
            y_pred.append(predicted_label)

        return y_true, y_pred

    def validate(self, df, targets):
        """
        對多個目標進行驗證
        """
        results = {}

        for target in targets:
            y_true, y_pred = self.validate_step(
                df, target, start_idx=self.window_size, end_idx=len(df)
            )

            accuracy = accuracy_score(y_true, y_pred)
            class_report = classification_report(y_true, y_pred)

            print(f"\nValidation results for {target}:")
            print(f"Accuracy: {accuracy:.4f}")
            print("Detailed Classification Report:")
            print(class_report)

            results[target] = {
                "accuracy": accuracy,
                "classification_report": class_report,
                "predictions": y_pred,
            }

        return results

    def predict_step(self, df, start_idx, end_idx):
        """
        逐步預測，確保只使用歷史數據
        """
        predictions = []

        for i in range(start_idx, end_idx):
            X_test = (
                df[self.features]
                .iloc[i - self.window_size : i]
                .values.flatten()
                .reshape(1, -1)
            )

            pred_proba = self.model.predict(X_test)
            pred_class = np.argmax(pred_proba, axis=1)[0]
            predicted_label = self.label_encoder.inverse_transform([pred_class])[0]

            predictions.append(predicted_label)

        return predictions

    def predict(self, df, targets):
        """
        對多個目標進行預測
        """
        if self.model is None:
            raise ValueError("模型尚未訓練，請先調用train方法")

        predictions = {}

        for target in targets:
            pred_list = self.predict_step(
                df, start_idx=self.window_size, end_idx=len(df)
            )

            predictions[target] = pred_list

            if target in df.columns:
                true_values = df[target].iloc[self.window_size : len(df)].values
                accuracy = accuracy_score(true_values, pred_list)
                class_report = classification_report(true_values, pred_list)

                print(f"\nPrediction results for {target}:")
                print(f"Accuracy: {accuracy:.4f}")
                print("Detailed Classification Report:")
                print(class_report)

        return predictions

In [ ]:
# 初始化分類器
classifier = TimeSeriesClassifier(window_size=10)
target_n=4
# 使用多執行緒進行參數調優
train_end_idx = int(len(df_train) * 0.8)
results_df = classifier.tune_parameters(
    df_train,
    features,
    targets[target_n],
    val_start_idx=train_end_idx,
    n_jobs=4,  # 使用4個執行緒，設為-1表示使用所有可用CPU
)

# 使用最佳參數訓練模型
classifier.train(df_train, features, targets[target_n], val_start_idx=train_end_idx)

# 驗證模型
validation_results = classifier.validate(
    df_validation, targets[target_n : target_n + 1]
)

# 預測（如果需要）
# predictions = classifier.predict(df_predict, targets)

# 查看參數調優結果
print("\nParameter tuning results:")
print(results_df.sort_values("score", ascending=False).head())

Training until validation scores don't improve for 30 rounds
Training until validation scores don't improve for 30 rounds
Training until validation scores don't improve for 30 rounds
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[11]	training's multi_logloss: 0.979131	valid_1's multi_logloss: 1.12964
Early stopping, best iteration is:
[10]	training's multi_logloss: 0.986817	valid_1's multi_logloss: 1.12921

New best score: 0.3230
Parameters: {'learning_rate': 0.01, 'num_leaves': 31, 'min_child_samples': 20, 'subsample': 0.8, 'colsample_bytree': 0.9}
Training until validation scores don't improve for 30 rounds
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[16]	training's multi_logloss: 0.944241	valid_1's multi_logloss: 1.1312
Early stopping, best iteration is:
[16]	training's multi_logloss: 0.944241	valid_1's multi_logloss: 1.1312

New best score: 0.3555
Parameters: {'learning_rate': 0.01